# testing score in link prediction

In [12]:
from n2i.__main__ import n2i_main, n2i_nx_graph
from n2i.node2vec import read_graph
import tensorflow as tf

In [2]:
graph_path = '../data/graph/lesmiserables/lesmiserables_edgelist.txt'

graph = read_graph(weighted=False, 
                   graph=graph_path,
                   directed=False)

In [3]:
emb = n2i_nx_graph(nx_graph=graph, topics=3, use_tf=False)

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [19]:
tf.reset_default_graph()
emb = n2i_nx_graph(nx_graph=graph, topics=2, use_tf=True, seed=10, translate=False, reduce=False, 
                   window_size=10,
                   walk_length=100,
                   num_walks=30,
                   dimensions=2,
                   beta=10)

mu:  (2,) log_std:  (2,)


In [20]:
X = []
y = []
for u, v in graph.edges():
    arr_u = list(emb[u])
    arr_v = list(emb[v])
    X.append(arr_u + arr_v)
    y.append(1)

In [21]:
import itertools as IT
import random

missing = [pair for pair in IT.combinations(graph.nodes(), 2) if not graph.has_edge(*pair)]
no_edges = random.choices(missing, k=len(graph.edges()))
for u, v in no_edges:
    arr_u = list(emb[u])
    arr_v = list(emb[v])
    X.append(arr_u + arr_v)
    y.append(0)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [23]:
# Create one-vs-rest logistic regression object
clf = LogisticRegression(random_state=0, multi_class='ovr')
# Train model
model = clf.fit(X_train, y_train)

In [24]:
model.score(X_test, y_test)

0.7619047619047619